<a href="https://colab.research.google.com/github/cosimo-schiavoni/Massive_Data_Project/blob/main/Downloads_TFOS4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
!wget -q https://www-eu.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install findspark
!pip install pyspark
!pip install -q kaggle
!pip install tensorflowonspark
!pip install sparkdl
!pip install tensorframes
!pip install petastorm

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 46.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=14ef325fff5451b8cfa1c37ec352744f7650801a996e4a8def08b788e4641d54
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 45 kB 1.6 MB/s 
     |████████████████████████████████| 99 kB 4.0 MB/s 
     |████████████████████████████████| 284 kB 4.9 MB/s 
     |████████████████████████████████| 136 kB 48.0 MB/s 
     |████████████████████████████████| 44 kB 2.7 MB/s 


In [6]:
!ls /content/spark-3.2.1-bin-hadoop2.7/

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [7]:
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7/"

In [66]:
# def main():
def main_fun(argv, ctx):
  import tensorflow as tf

if __name__ == '__main__':
  # tf.app.run()
  #Spark
  import pyspark
  from pyspark.sql import SparkSession
  from pyspark import SparkContext, SparkConf
  from pyspark.sql import SQLContext
  from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
  from pyspark.ml import Pipeline
  from pyspark.sql.functions import rand
  from pyspark.mllib.evaluation import MulticlassMetrics
  # Keras / Deep Learning
  import tensorflow as tf
  from tensorflowonspark import TFParallel
  from tensorflowonspark import TFCluster
  import argparse
  from keras.models import Sequential
  from keras.layers.core import Dense, Dropout, Activation
  from keras import optimizers, regularizers
  from tensorflow.keras.optimizers import Adam
  #from my model
  import matplotlib
  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg
  import keras
  from tensorflow.keras import layers
  from tensorflow.keras.models import Model
  import pickle
  import shutil
  import random
  import skimage.io as io
  from copy import deepcopy
  import tensorflow_datasets as tfds
  import os
  from functools import reduce
  from google.colab import files
  import os
  import zipfile
  import numpy as np
  import tensorflow_datasets as tfds
  import pathlib
  
  sc = SparkContext(conf=SparkConf().setAppName("TFOS"))
  executors = sc._conf.get("spark.executor.instances")
  num_executors = int(executors) if executors is not None else 1

  parser = argparse.ArgumentParser()
  parser.add_argument("--cluster_size", help="number of nodes in the cluster (for S with labelspark Standalone)", type=int, default=num_executors)
  parser.add_argument('--images_labels', type=str, help='Directory for input images with labels')
  parser.add_argument("--export_dir", help="HDFS path to export model", type=str, default="mnist_export")
  parser.add_argument("--output", help="HDFS path to save predictions", type=str, default="predictions")
  args, _ = parser.parse_known_args()
  print("args: {}".format(args))


ValueError: ignored

In [3]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

Saving kaggle.json to kaggle.json


In [4]:
os.environ["data_source"] = "./Face_Comics_data"

api = KaggleApi()
api.authenticate()

if not os.path.exists(os.environ["data_source"]):
  os.makedirs(os.environ["data_source"])
  if "comic-faces-paired-synthetic-v2" not in os.listdir():
    if "comic-faces-paired-synthetic-v2.zip" not in os.listdir():
      ! kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2
      with zipfile.ZipFile("comic-faces-paired-synthetic-v2.zip", 'r') as f:
        f.extractall("comic-faces-paired-synthetic-v2")
    os.remove("comic-faces-paired-synthetic-v2.zip")


!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/comics" "./Face_Comics_data"
!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/faces" "./Face_Comics_data"

!rm -rf comic-faces-paired-synthetic-v2

100% 2.17G/2.18G [00:16<00:00, 157MB/s]
100% 2.18G/2.18G [00:16<00:00, 146MB/s]


In [ ]:
#strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

NameError: ignored

In [ ]:
"""
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
"""

In [67]:
data_dir= "./Face_Comics_data"

tf.random.set_seed(123456)

EPOCHS = 15
BATCH_SIZE = 32
IMG_SIZE = (350, 350)

os.listdir(data_dir)

['faces', 'comics']

In [63]:
#data augmentation

@tf.function
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (180-x)
  else:
    x
  return x

###Randim invert
#@tf.function 
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

#@tf.function 
class RandomInvert(layers.Layer):
  #@tf.function 
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor 
###Start added code
  def get_config(self):
    config = {
        "factor" : self.factor
        }
    return config
###  End        
  #@tf.function 
  def call(self, x):
    return random_invert_img(x)



###Kernel Inizializer Sobel_x
@tf.function 
def hedge_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [-5, -4, 0, 4, 5], 
            [-8, -10, 0, 10, 8], 
            [-10, -20, 0, 20, 10], 
            [-8, -10, 0, 10, 8], 
            [-5, -4, 0, 4, 5]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (5, 5, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

@tf.function 
def vertical_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 0, -1], 
            [1, 0, -1], 
            [1, 0, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

@tf.function 
def horizontal_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 1, 1], 
            [0, 0, 0], 
            [-1, -1, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

In [68]:

###Create CNN
#Initialize the CNN
cnn = tf.keras.models.Sequential()
   
cnn.add(tf.keras.Sequential([
	layers.RandomFlip("horizontal_and_vertical"),
	layers.RandomRotation(0.3),
    layers.RandomContrast(0.5, seed=None),
    #RandomInvert(),
    layers.RandomZoom(height_factor=(-0.2, +0.3)),
    layers.RandomTranslation(height_factor=(-0.2, +0.3),width_factor=(-0.2, +0.3)),
    layers.Rescaling(1./255, offset= -1)
    ]))

#DROPOUT
#Introduce the Convolution layer with Kernel_initializer=Sobel_x
#Convolution
cnn.add(tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, activation='relu'))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Convolution
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Second Convolutional Layer
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
#Third Convolutional Layer
#cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, activation = 'relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))
#Flattening
cnn.add(tf.keras.layers.Flatten())
#Full Connection
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
#DROPOUTS ok(0.1)
cnn.add(tf.keras.layers.Dropout(0.1))
#Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
###Training CNN
#Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#training the CNN on the train_dataset and validate it on the validation_dataset
#model_fit = cnn.fit(x = train_dataset, validation_data = validation_dataset, epochs = EPOCHS)

cnn.build(input_shape = [None,350,350,3]) # `input_shape` is the shape of the input data

cnn.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_8 (Sequential)   (None, 350, 350, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 173, 173, 32)      2432      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 86, 86, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 42, 42, 32)        9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 21, 21, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 10, 10, 32)       

***estimator.train - DEFINE INPUT FUNCTION***


In [77]:
"""#estimator.train 
def input_fn():
  # example of progressively loading images from file
  from keras.preprocessing.image import ImageDataGenerator
  # create generator
  datagen = ImageDataGenerator()
  # prepare an iterators for each dataset
  train_it = datagen.flow_from_directory('./Face_Comics_data', class_mode='binary',subset='training', color_mode="rgb",  batch_size=BATCH_SIZE, target_size=IMG_SIZE,)
  val_it = datagen.flow_from_directory('./Face_Comics_data', class_mode='binary',subset='validation', color_mode="rgb",  batch_size=BATCH_SIZE, target_size=IMG_SIZE,)
  #val_it = datagen.flow_from_directory('data/validation/', class_mode='binary')
  #test_it = datagen.flow_from_directory('data/test/', class_mode='binary')
  # confirm the iterator works
  features, labels = train_it.next()
  print('Batch shape=%s, min=%.3f, max=%.3f' % (features.shape, features.min(), features.max()))
  return features, labels
"""



***CREATE THE ESTIMATOR***

In [65]:
"""
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=cnn, model_dir=model_dir)
"""

INFO:tensorflow:Using default config.


2022-05-19 10:29:29,869 INFO (MainThread-61) Using default config.


INFO:tensorflow:Using the Keras model provided.


2022-05-19 10:29:29,875 INFO (MainThread-61) Using the Keras model provided.
/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


ValueError: ignored

***TRAIN METHOD***

In [70]:
"""
keras_estimator.train(input_fn=input_fn, steps=500)
#tf.estimator.train(input_fn=input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10)
print('Eval result: {}'.format(eval_result))
"""

Found 20000 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Batch shape=(32, 350, 350, 3), min=0.000, max=255.000
INFO:tensorflow:Calling model_fn.


2022-05-19 12:43:37,464 INFO (MainThread-61) Calling model_fn.
/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Done calling model_fn.


2022-05-19 12:43:38,947 INFO (MainThread-61) Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 500...


2022-05-19 12:43:51,409 INFO (MainThread-61) Calling checkpoint listeners after saving checkpoint 500...


INFO:tensorflow:loss = 0.12546581, step = 500


2022-05-19 12:43:54,489 INFO (MainThread-61) loss = 0.12546581, step = 500


KeyboardInterrupt: ignored

***estimator.train_and_evaluate - TRAIN AND EVALUATE METHOD***

In [6]:
#estimator.train_and_evaluate
from keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(
rescale=1./255,
#shear_range=0.2,
#zoom_range=0.2,
#horizontal_flip=True,
validation_split=0.2)
  
def train_input_fn(): 
    train_it = datagen.flow_from_directory(
    './Face_Comics_data', 
    class_mode='binary',
    subset='training', 
    color_mode="rgb", 
    shuffle=True,
    seed=123456,
    batch_size=BATCH_SIZE, 
    target_size=IMG_SIZE
    )
    features, labels = train_it.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (features.shape, features.min(), features.max()))
    return features, labels


def eval_input_fn(): # returns x, y
    val_it = datagen.flow_from_directory(
    './Face_Comics_data', 
    class_mode='binary',
    subset='validation', 
    color_mode="rgb", 
    shuffle=True,
    seed=123456,
    batch_size=BATCH_SIZE, 
    target_size=IMG_SIZE
    )
    features, labels = val_it.next()
    print('Batch shape=%s, min=%.3f, max=%.3f' % (features.shape, features.min(), features.max()))
    return features, labels


INFO:tensorflow:Not using Distribute Coordinator.


2022-05-19 02:18:19,578 INFO (MainThread-1019) Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


2022-05-19 02:18:19,584 INFO (MainThread-1019) Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.


...

INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpx2__w9lm/model.ckpt.


2022-05-19 02:18:22,857 INFO (MainThread-1019) Saving checkpoints for 0 into /tmp/tmpx2__w9lm/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


2022-05-19 02:18:22,985 INFO (MainThread-1019) Calling checkpoint listeners after saving checkpoint 0...


DataLossError: ignored

***CREATE THE ESTIMATOR***

In [ ]:
import tempfile
model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=cnn, model_dir=model_dir)

***TRAIN AND EVALUATE***

In [78]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)

tf.estimator.train_and_evaluate(keras_estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.


2022-05-19 13:07:15,338 INFO (MainThread-61) Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


2022-05-19 13:07:15,346 INFO (MainThread-61) Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpm3snpvmf/model.ckpt-1000


2022-05-19 13:12:42,217 INFO (MainThread-61) Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpm3snpvmf/model.ckpt-1000


INFO:tensorflow:Loss for final step: 0.54626036.


2022-05-19 13:12:42,254 INFO (MainThread-61) Loss for final step: 0.54626036.


({'accuracy': 0.4375, 'global_step': 1000, 'loss': 0.9305346}, [])

In [80]:

# Running single-node TF instances on each executor
#TFParallel.run(sc, inference, args, args.cluster_size)

cluster = TFCluster.run(sc, main_fun, args, args.cluster_size, args.num_ps, args.tensorboard, TFCluster.InputMode.TENSORFLOW)
 #cluster.shutdown()

AttributeError: ignored

In [ ]:
"""
  parser = argparse.ArgumentParser()
  parser.add_argument("--cluster_size", help="number of nodes in the cluster (for S with labelspark Standalone)", type=int, default=num_executors)
  parser.add_argument('--images_labels', type=str, help='Directory for input images with labels')
  parser.add_argument("--export_dir", help="HDFS path to export model", type=str, default="mnist_export")
  parser.add_argument("--output", help="HDFS path to save predictions", type=str, default="predictions")
  args, _ = parser.parse_known_args()
  print("args: {}".format(args))
"""

In [1]:
# def main():
def main_fun(argv, ctx):
  import tensorflow as tf

if __name__ == '__main__':
  # tf.app.run()
  #Spark
  import pyspark
  from pyspark.sql import SparkSession
  from pyspark import SparkContext, SparkConf
  from pyspark.sql import SQLContext
  from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
  from pyspark.ml import Pipeline
  from pyspark.sql.functions import rand
  from pyspark.mllib.evaluation import MulticlassMetrics
  # Keras / Deep Learning
  import tensorflow as tf
  from tensorflowonspark import TFParallel
  from tensorflowonspark import TFCluster
  import argparse
  from keras.models import Sequential
  from keras.layers.core import Dense, Dropout, Activation
  from keras import optimizers, regularizers
  from tensorflow.keras.optimizers import Adam
  #from my model
  import matplotlib
  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg
  import keras
  from tensorflow.keras import layers
  from tensorflow.keras.models import Model
  import pickle
  import shutil
  import random
  import skimage.io as io
  from copy import deepcopy
  import tensorflow_datasets as tfds
  import os
  from functools import reduce
  from google.colab import files
  import os
  import zipfile
  import numpy as np
  import tensorflow_datasets as tfds
  import pathlib
  
  sc = SparkContext(conf=SparkConf().setAppName("TFOS"))
  executors = sc._conf.get("spark.executor.instances")
  num_executors = int(executors) if executors is not None else 1

  parser = argparse.ArgumentParser()
  parser.add_argument("--cluster_size", help="number of nodes in the cluster (for Spark Standalone)", type=int, default=num_executors)
  parser.add_argument("--num_ps", help="number of parameter servers", type=int, default=1)
  #parser.add_argument("--tensorboard", help="launch tensorboard process", action="store_true")
  args = parser.parse_args()
  print("args: {}".format(args))

  data_dir= "./Face_Comics_data"

  tf.random.set_seed(123456)

  EPOCHS = 15
  BATCH_SIZE = 32
  IMG_SIZE = (350, 350)

  os.listdir(data_dir)

  #data augmentation

  @tf.function
  def random_invert_img(x, p=0.5):
    if  tf.random.uniform([]) < p:
      x = (180-x)
    else:
      x
    return x

  ###Randim invert
  #@tf.function 
  def random_invert(factor=0.5):
    return layers.Lambda(lambda x: random_invert_img(x, factor))

  #@tf.function 
  class RandomInvert(layers.Layer):
    #@tf.function 
    def __init__(self, factor=0.5, **kwargs):
      super().__init__(**kwargs)
      self.factor = factor 
  ###Start added code
    def get_config(self):
      config = {
          "factor" : self.factor
          }
      return config
  ###  End        
    #@tf.function 
    def call(self, x):
      return random_invert_img(x)



  ###Kernel Inizializer Sobel_x
  @tf.function 
  def hedge_detector(shape, dtype=None):
      print(shape)    
      sobel_x = tf.constant(
          [
              [-5, -4, 0, 4, 5], 
              [-8, -10, 0, 10, 8], 
              [-10, -20, 0, 20, 10], 
              [-8, -10, 0, 10, 8], 
              [-5, -4, 0, 4, 5]
          ], dtype=dtype )
      #create the missing dims.
      sobel_x = tf.reshape(sobel_x, (5, 5, 1, 1))

      print(tf.shape(sobel_x))
      #tile the last 2 axis to get the expected dims.
      sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

      print(tf.shape(sobel_x))
      return sobel_x

  @tf.function 
  def vertical_detector(shape, dtype=None):
      print(shape)    
      sobel_x = tf.constant(
          [
              [1, 0, -1], 
              [1, 0, -1], 
              [1, 0, -1]
          ], dtype=dtype )
      #create the missing dims.
      sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

      print(tf.shape(sobel_x))
      #tile the last 2 axis to get the expected dims.
      sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

      print(tf.shape(sobel_x))
      return sobel_x

  @tf.function 
  def horizontal_detector(shape, dtype=None):
      print(shape)    
      sobel_x = tf.constant(
          [
              [1, 1, 1], 
              [0, 0, 0], 
              [-1, -1, -1]
          ], dtype=dtype )
      #create the missing dims.
      sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

      print(tf.shape(sobel_x))
      #tile the last 2 axis to get the expected dims.
      sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

      print(tf.shape(sobel_x))
      return sobel_x

  ###Create CNN
  #Initialize the CNN
  cnn = tf.keras.models.Sequential()
    
  cnn.add(tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
      layers.RandomContrast(0.5, seed=None),
      #RandomInvert(),
      layers.RandomZoom(height_factor=(-0.2, +0.3)),
      layers.RandomTranslation(height_factor=(-0.2, +0.3),width_factor=(-0.2, +0.3)),
      layers.Rescaling(1./255, offset= -1)
      ]))

  #DROPOUT
  #Introduce the Convolution layer with Kernel_initializer=Sobel_x
  #Convolution
  cnn.add(tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, activation='relu'))
  #Pooling
  cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
  #Convolution
  cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
  #Pooling
  cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
  #Second Convolutional Layer
  cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
  cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
  #Third Convolutional Layer
  #cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, activation = 'relu'))
  #cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))
  #Flattening
  cnn.add(tf.keras.layers.Flatten())
  #Full Connection
  cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
  #DROPOUTS ok(0.1)
  cnn.add(tf.keras.layers.Dropout(0.1))
  #Output Layer
  cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
  ###Training CNN
  #Compiling the CNN
  cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  #training the CNN on the train_dataset and validate it on the validation_dataset
  #model_fit = cnn.fit(x = train_dataset, validation_data = validation_dataset, epochs = EPOCHS)

  cnn.build(input_shape = [None,350,350,3]) # `input_shape` is the shape of the input data

  cnn.summary()

  #estimator.train_and_evaluate
  from keras.preprocessing.image import ImageDataGenerator
  # create generator
  datagen = ImageDataGenerator(
  rescale=1./255,
  #shear_range=0.2,
  #zoom_range=0.2,
  #horizontal_flip=True,
  validation_split=0.2)

  @tf.function     
  def train_input_fn(): 
      train_it = datagen.flow_from_directory(
      './Face_Comics_data', 
      class_mode='binary',
      subset='training', 
      color_mode="rgb", 
      shuffle=True,
      seed=123456,
      batch_size=BATCH_SIZE, 
      target_size=IMG_SIZE
      )
      features, labels = train_it.next()
      print('Batch shape=%s, min=%.3f, max=%.3f' % (features.shape, features.min(), features.max()))
      return features, labels

  @tf.function 
  def eval_input_fn(): # returns x, y
      val_it = datagen.flow_from_directory(
      './Face_Comics_data', 
      class_mode='binary',
      subset='validation', 
      color_mode="rgb", 
      shuffle=True,
      seed=123456,
      batch_size=BATCH_SIZE, 
      target_size=IMG_SIZE
      )
      features, labels = val_it.next()
      print('Batch shape=%s, min=%.3f, max=%.3f' % (features.shape, features.min(), features.max()))
      return features, labels

  import tempfile
  model_dir = tempfile.mkdtemp()
  keras_estimator = tf.keras.estimator.model_to_estimator(
      keras_model=cnn, model_dir=model_dir)

  TFCluster.run(sc, main_fun, args, args.cluster_size, args.num_ps, TFCluster.InputMode.TENSORFLOW)


usage: ipykernel_launcher.py [-h] [--cluster_size CLUSTER_SIZE]
                             [--num_ps NUM_PS]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-b90280e7-9823-4d53-a550-9356be82d0d8.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
